In [3]:
!kaggle competitions download -c titanic

100%|███████████████████████████████████████| 34.1k/34.1k [00:00<00:00, 347kB/s]
100%|███████████████████████████████████████| 34.1k/34.1k [00:00<00:00, 345kB/s]


In [4]:
from zipfile import ZipFile
with ZipFile('titanic.zip', 'r') as zipObj:
    zipObj.extractall()

In [479]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
gender_submission = pd.read_csv('gender_submission.csv')

In [480]:
# target = train['Survived']
# train = train.drop('Survived', axis=1)

In [482]:
train.dtypes.value_counts()

int64      5
object     5
float64    2
dtype: int64

In [483]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [490]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.preprocessing import OneHotEncoder
from copy import deepcopy
from pdb import set_trace as breakpoint

class Transformer(BaseEstimator, TransformerMixin):
    def __init__(self, impute_strategy='mean', cat_features=None, num_features=None):
        self.imputer = SimpleImputer(strategy=impute_strategy)
        self.missing_indicator = MissingIndicator()
        self.cat_features = cat_features
        self.num_features = num_features


    def fit_transform(self, X_train):
        X_train = deepcopy(X_train)
        X_train = X_train.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip())
#         X_train = X_train.drop('Name', axis=1)
        missing = missing_indicator.fit_transform(X_train)
        self.detect_features(X_train)
        X_train[self.num_features] = self.imputer.fit_transform(X_train[self.num_features]).copy()
        X_train = self.one_hot_encode(X_train)
        self.columns = X_train.columns
        return X_train


    def transform(self, X_test):
        X_test = deepcopy(X_test)
        X_test = X_test.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip())
#         X_test = X_test.drop('Name', axis=1)
        X_test[self.num_features] = self.imputer.transform(X_test[self.num_features])
        X_test = self.one_hot_encode(X_test)
        X_test = self.align(X_test)
        return X_test
    
    
    def one_hot_encode(self, X):
        X = deepcopy(X)
        X = pd.get_dummies(X, columns=self.cat_features)
        return X

    
    def align(self, X_test):
        X_test = deepcopy(X_test)
        cols_to_drop = list(set(X_test.columns) - set(self.columns))
        cols_to_add = list(set(self.columns) - set(X_test.columns))
        X_test = X_test.drop(cols_to_drop, axis=1)
        X_test = X_test.assign(**dict(zip(cols_to_add, [0] * len(cols_to_add))))
        X_test = X_test[self.columns]
        return X_test


    def detect_features(self, X_train):
        self.cat_features = [col for col in X_train.columns if X_train[col].dtype=='object']
        self.num_features = [col for col in X_train.columns if col not in self.cat_features]

In [491]:
columns = {}
for col in transformer.columns:
    columns[col] = 0
    
for col in transformer.columns:
    columns[col] += 1

AttributeError: 'Transformer' object has no attribute 'columns'

In [492]:
[col for col in columns.keys() if columns[col] > 1]

[]

In [493]:
y = train['Survived']
X = train.drop('Survived', axis=1)
X = X.set_index('PassengerId')
X_test = test.set_index('PassengerId')

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y)

transformer = Transformer()
X = transformer.fit_transform(X)
# X_train = transformer.fit_transform(X_train)
# X_val = transformer.transform(X_val)
X_test = transformer.transform(X_test)

ValueError: Expected 2D array, got 1D array instead:
array=['Mr' 'Mrs' 'Miss' 'Mrs' 'Mr' 'Mr' 'Mr' 'Master' 'Mrs' 'Mrs' 'Miss' 'Miss'
 'Mr' 'Mr' 'Miss' 'Mrs' 'Master' 'Mr' 'Mrs' 'Mrs' 'Mr' 'Mr' 'Miss' 'Mr'
 'Miss' 'Mrs' 'Mr' 'Mr' 'Miss' 'Mr' 'Don' 'Mrs' 'Miss' 'Mr' 'Mr' 'Mr' 'Mr'
 'Mr' 'Miss' 'Miss' 'Mrs' 'Mrs' 'Mr' 'Miss' 'Miss' 'Mr' 'Mr' 'Miss' 'Mr'
 'Mrs' 'Master' 'Mr' 'Mrs' 'Mrs' 'Mr' 'Mr' 'Miss' 'Mr' 'Miss' 'Master'
 'Mr' 'Miss' 'Mr' 'Master' 'Mr' 'Master' 'Mrs' 'Mr' 'Miss' 'Mr' 'Mr'
 'Miss' 'Mr' 'Mr' 'Mr' 'Mr' 'Mr' 'Mr' 'Master' 'Miss' 'Mr' 'Mr' 'Miss'
 'Mr' 'Miss' 'Mrs' 'Mr' 'Mr' 'Miss' 'Mr' 'Mr' 'Mr' 'Mr' 'Mr' 'Mr' 'Mr'
 'Mr' 'Mr' 'Mrs' 'Mr' 'Miss' 'Mr' 'Mr' 'Mr' 'Mr' 'Mr' 'Miss' 'Mr' 'Mr'
 'Miss' 'Mr' 'Miss' 'Mr' 'Miss' 'Miss' 'Mr' 'Mr' 'Mr' 'Mr' 'Miss' 'Mr'
 'Mr' 'Mr' 'Miss' 'Mr' 'Master' 'Mr' 'Mr' 'Miss' 'Mr' 'Mr' 'Mr' 'Mrs'
 'Mrs' 'Mr' 'Mr' 'Miss' 'Mr' 'Mr' 'Mr' 'Mrs' 'Miss' 'Mrs' 'Mr' 'Mr' 'Mr'
 'Mr' 'Miss' 'Mr' 'Rev' 'Rev' 'Mrs' 'Mr' 'Mr' 'Mr' 'Mr' 'Miss' 'Mr' 'Mr'
 'Mast

In [409]:
rf = RandomForestClassifier()

In [410]:
rf.fit(X, y)

/Users/abdul/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [430]:
from xgboost import XGBClassifier

In [431]:
xgb = XGBClassifier()

In [432]:
cross_val_score(xgb, X, y)

/Users/abdul/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


array([0.79461279, 0.81818182, 0.82491582])

In [433]:
xgb.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [434]:
gender_submission['Survived'] = xgb.predict(X_test)

In [435]:
gender_submission.to_csv('submission.csv', index=False)

In [436]:
!kaggle competitions submit -c titanic -f submission.csv -m "Message"

100%|████████████████████████████████████████| 2.77k/2.77k [00:08<00:00, 351B/s]
Successfully submitted to Titanic: Machine Learning from Disaster